## Import libraries

In [ ]:
import pandas as pd
import datacube
from datacube.storage import masking
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
import numpy as np

# Load utility functions
from utils.DEADataHandling import load_clearsentinel2, load_clearlandsat

## Import yield data

Need more localised/specific data. Currently using harvest values for all of NSW for all winter crops.

In [ ]:
yield_nsw = pd.read_csv("ancillary_data/NSW_Yield_Data.csv", parse_dates=['time'])

In [ ]:
print(yield_nsw.head())

## Load Landsat geomedian for Harden, NSW

Need higher-cadence data. Currently using geomedian for whole year over Harden, which is primarily farmland, but not entirely. Need to find a way to exclude non-farm regions.

In [ ]:
harden_lat = (-34.7, -34.6)
harden_lon = (148.3, 148.4)
time_range = (1989, 2017)

In [ ]:
# Connect to the datacube
dc = datacube.Datacube(app='index-insurance')

# Create the 'query' dictionary object, which contains the longitudes, latitudes and time provided above
query = {
    'y': harden_lat,
    'x': harden_lon,
    'time': time_range,
    'output_crs': 'EPSG:28352',
    'resolution': (-25, 25)
}

In [ ]:
# Load Landsat 8 data for the time and area in the query. This may take several minutes, so please be patient.
landsat8_ds = dc.load(
    product='ls8_nbart_geomedian_annual',
    **query,
    measurements=['red', 'green', 'blue', 'nir']
)

# Load Landsat 7 data for the time and area in the query. This may take several minutes, so please be patient.
landsat7_ds = dc.load(
    product='ls7_nbart_geomedian_annual',
    **query,
    measurements=['red', 'green', 'blue', 'nir']
)

# Load Landsat 5 data for the time and area in the query. This may take several minutes, so please be patient.
landsat5_ds = dc.load(
    product='ls5_nbart_geomedian_annual',
    **query,
    measurements=['red', 'green', 'blue', 'nir']
)

## Filter and combine Landsat datasets
Try and remove Landsat images affected by SLC error (Landsat 7). 

In [ ]:
landsat5_filtered_1 = landsat5_ds.sel(time=landsat5_ds.time < np.datetime64('2001-01-01'))
landsat5_filtered_2 = landsat5_ds.sel(time=landsat5_ds.time > np.datetime64('2003-01-01'))
landsat7_filtered = landsat7_ds.sel(time=landsat7_ds.time < np.datetime64('2004-01-01'))

landsat_combined = xr.concat([landsat5_filtered_1, landsat7_filtered, landsat5_filtered_2, landsat8_ds], dim='time')

## Calculate NDVI for combined Landsat data

In [ ]:
landsat_combined['ndvi'] = (landsat_combined.nir - landsat_combined.red)/(landsat_combined.nir + landsat_combined.red)
landsat_mean = landsat_combined.mean(dim=['x','y'])

ndvi_df = landsat_mean.ndvi.to_dataframe()

ndvi_yield = pd.merge(yield_nsw, ndvi_df, on='time')
print(landsat_combined)

## Plot Yield vs. NDVI

In [ ]:
ndvi_yield.plot.scatter(x='ndvi', y='Yield_t_per_hectare')
plt.xlabel('NDVI')
plt.ylabel('Yield (t/ha)')
plt.show()

## Fit linear regression to Yield vs. NDVI

In [ ]:
from sklearn import linear_model

X = np.asarray(ndvi_yield['ndvi']).reshape(-1, 1)
y = ndvi_yield['Yield_t_per_hectare']

lm = linear_model.LinearRegression()
model = lm.fit(X,y)

predictions = lm.predict(X)

In [ ]:
ndvi_yield.plot.scatter(x='ndvi', y='Yield_t_per_hectare')
plt.plot(X, predictions)
plt.xlabel('NDVI')
plt.ylabel('Yield (t/ha)')
plt.show()

In [ ]:
print(lm.score(X,y))

## Plot distribution of NDVI

In [ ]:
ndvi_yield['ndvi'].hist(bins=10)
plt.xlabel('NDVI')
plt.show()

## Plot distribution of Yield

In [ ]:
ndvi_yield['Yield_t_per_hectare'].hist(bins=10)
plt.xlabel('Yield (t/ha)')
plt.show()

## Load Sentinel 2 for Harden

In [ ]:
# Specify the product measurments to load from Sentinel-2
measurements = (
    'nbar_red',
    'nbar_green',
    'nbar_blue',
    'nbar_nir_1',
)

# Specify the minimum proportion of good quality pixels for an image.
# The image will be excluded if masking results in fewer pixels than
# the set proportion.
# Setting this to 0.0 includes all images
min_good_pixel_prop = 0.5

# Load the data and mask out bad quality pixels
ds_s2 = load_clearsentinel2(
    dc=dc,
    query=query,
    sensors=['s2a', 's2b'],
    product='ard',
    bands_of_interest=measurements,
    masked_prop=min_good_pixel_prop
)


# sentinel_2a_ds = dc.load(
#     product='s2a_ard_granule',
#     **query,
#     measurements=['nbar_red', 'nbar_green', 'nbar_blue', 'nbar_nir_1'],
#     group_by='solar_day'
# )
# print(sentinel_2a_ds)

# sentinel_2b_ds = dc.load(
#     product='s2b_ard_granule',
#     **query,
#     measurements=['nbar_red', 'nbar_green', 'nbar_blue', 'nbar_nir_1'],
#     group_by='solar_day'
# )
# print(sentinel_2b_ds)

# sentinel_2_ds = xr.concat([sentinel_2a_ds, sentinel_2b_ds], dim='time')
# print(sentinel_2_ds)

In [ ]:
from utils.utils import three_band_image

time = 10
time_string = str(ds_s2.time.isel(time=time).values).split('.')[0]

test = three_band_image(ds_s2, ['nbar_red', 'nbar_green', 'nbar_blue'], time=time, figsize=[20,20])
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_title("Timestep {}".format(time_string), fontweight='bold', fontsize=16)
ax.set_xticklabels(ds_s2.x.values)
ax.set_yticklabels(ds_s2.y.values)
ax.set_xlabel('Easting', fontweight='bold')
ax.set_ylabel('Northing', fontweight='bold')

plt.imshow(test)

In [ ]:
ds_s2['ndvi'] = (ds_s2.nbar_nir_1 - ds_s2.nbar_red)/(ds_s2.nbar_nir_1 + ds_s2.nbar_red)
ds_s2_mean = ds_s2.mean(dim=['x','y'])

s2_ndvi_df = ds_s2.ndvi.to_dataframe()

In [ ]:
ds_s2_mean.ndvi.plot()

## Load LS8 Fractional Cover for Harden

Load photosynthetic vegetation measurement from Landsat 5,8 (FC currently unavailable in sandbox for Landsat 7). Also load WoFS feature layers to make mask. 

In [ ]:
ls8_fc_ds = dc.load(
    product='ls8_fc_albers',
    **query,
    measurements=['PV']
)

ls5_fc_ds = dc.load(
    product='ls5_fc_albers',
    **query,
    measurements=['PV']
)

ls_fc_combined_ds = xr.concat([ls5_fc_ds, ls8_fc_ds], dim='time')

wofls_ds = dc.load(product='wofs_albers', **query)

Make mask based on which pixels are classified as dry.

In [ ]:
wofl_mask = masking.make_mask(wofls_ds, dry=True)

Apply mask to combined Landsat data

In [ ]:
ls_fc_combined_masked_mean = ls_fc_combined_masked_ds.mean(dim=['x','y'])

ls_fc_combined_masked_mean.PV.plot.line('b-', aspect=5, size=4)
plt.show()

In [ ]:
ls_fc_resampled = ls_fc_combined_masked_mean.resample(time='1W').interpolate('linear')

ls_fc_weekgroup_average = ls_fc_resampled.PV.groupby('time.week').mean()

fig, ax = plt.subplots(figsize=(12,4))
ls_fc_resampled_test.plot.line()
ax.axvspan(xmin=38, xmax=52, alpha=0.25, color='green', label="Harvest")
ax.axvspan(xmin=16, xmax=23, alpha=0.25, color='red', label="Sowing")
plt.legend()
plt.title("Weekly average PV over archive") 
plt.show()

In [ ]:
monthly = ls_fc_resampled[ls_fc_resampled.groupby('time.month').mean()
print(monthly)

In [ ]:
november_ds = ls_fc_resampled.where(ls_fc_resampled.time.dt.month == 11).dropna('time').groupby('time.year').mean()

yield_nsw['year'] = yield_nsw['time'].dt.year
test = yield_nsw.set_index(['year']).to_xarray()

november_pv_yield = xr.merge([november_ds, test], join='inner').to_dataframe()

In [ ]:
ax = november_pv_yield.plot.scatter(x='PV', y='Yield_t_per_hectare', figsize=(10,10))
for index, row in november_pv_yield.iterrows():
     ax.annotate(index, (row['PV'], row['Yield_t_per_hectare']))

ax.set_xlabel('PV')
ax.set_ylabel('Yield (t/ha)')
plt.show()

In [ ]:
X_pv = np.asarray(november_pv_yield['PV']).reshape(-1, 1)
y_yield = november_pv_yield['Yield_t_per_hectare']

lm_yield = linear_model.LinearRegression()
model_yield = lm_yield.fit(X_pv,y_yield)

predictions_yield = lm_yield.predict(X_pv)

print(lm_yield.score(X_pv,y_yield))

In [ ]:
november_pv_yield.plot.scatter(x='PV', y='Yield_t_per_hectare')
plt.plot(X_pv, predictions_yield)
plt.xlabel('NDVI')
plt.ylabel('Yield (t/ha)')
plt.show()